## 1. Configuration initiale

Importation des bibliothèques nécessaires et définition des paramètres de connexion à la base de données.

In [13]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from pathlib import Path
import time

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

Dossier des données transformées: C:\Users\Saint\IdeaProjects\ETL_Simple_Project\ETL_Projet_Pandemies\donnees\transformees


## 2. Configuration de la connexion à la base de données

Définition des paramètres de connexion à la base de données MySQL.

In [14]:
# Paramètres de connexion à la base de données
config_db = {
    "host": "localhost",
    "user": "root",
    "password": "",  # Laisser vide si aucun mot de passe
    "database": "epiviz",
    "port": 3306
}

print("Paramètres de connexion à la base de données:")
for cle, valeur in config_db.items():
    if cle != "password":
        print(f"- {cle}: {valeur}")
    else:
        print(f"- {cle}: {'(aucun)' if valeur == '' else '******'}")

Paramètres de connexion à la base de données:
- host: localhost
- user: root
- password: (aucun)
- database: epiviz
- port: 3306


## 3. Fonction de connexion à la base de données

Définition d'une fonction pour établir une connexion à la base de données MySQL.

In [15]:
def connecter_mysql(config=None, creer_db=False):
    """
    Établit une connexion à la base de données MySQL.
    
    Args:
        config (dict): Dictionnaire contenant les paramètres de connexion.
        creer_db (bool): Si True, crée la base de données si elle n'existe pas.
        
    Returns:
        tuple: (connexion, curseur) si la connexion est établie, (None, None) sinon.
    """
    if config is None:
        config = config_db.copy()
    
    try:
        # Si on veut créer la base de données, on se connecte sans spécifier la base de données
        if creer_db:
            config_sans_db = config.copy()
            database = config_sans_db.pop("database", None)
            connexion = mysql.connector.connect(**config_sans_db)
            curseur = connexion.cursor()
            
            # Création de la base de données si elle n'existe pas
            curseur.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
            curseur.execute(f"USE {database}")
            print(f"Base de données '{database}' créée ou sélectionnée avec succès.")
        else:
            # Connexion à la base de données existante
            connexion = mysql.connector.connect(**config)
            curseur = connexion.cursor()
            print(f"Connexion à la base de données '{config['database']}' établie avec succès.")
        
        return connexion, curseur
    
    except Error as e:
        print(f"Erreur lors de la connexion à la base de données: {e}")
        return None, None

## 4. Test de la connexion à la base de données

Test de la connexion à la base de données MySQL.

In [16]:
# Test de la connexion à la base de données
print("Test de la connexion à la base de données...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql(creer_db=True)
    
    if connexion is not None and curseur is not None:
        # Vérification de la version de MySQL
        curseur.execute("SELECT VERSION()")
        version = curseur.fetchone()
        print(f"Version de MySQL: {version[0]}")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("Connexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors du test de la connexion à la base de données: {e}")

Test de la connexion à la base de données...
Base de données 'epiviz' créée ou sélectionnée avec succès.
Version de MySQL: 10.4.27-MariaDB
Connexion fermée.


## 5. Définition du schéma de la base de données

Définition des requêtes SQL pour créer les tables de la base de données.

In [17]:
# Définition des requêtes SQL pour créer les tables
requetes_creation_tables = {
    "calendar": """
    CREATE TABLE IF NOT EXISTS calendar (
        id INT AUTO_INCREMENT PRIMARY KEY,
        date_value INT DEFAULT NULL
    ) ENGINE=InnoDB;
    """,
    
    "location": """
    CREATE TABLE IF NOT EXISTS location (
        id INT AUTO_INCREMENT PRIMARY KEY,
        country CHAR(50) NOT NULL DEFAULT '0',
        continent CHAR(50) NOT NULL DEFAULT '0'
    ) ENGINE=InnoDB;
    """,
    
    "pandemie": """
    CREATE TABLE IF NOT EXISTS pandemie (
        id INT AUTO_INCREMENT PRIMARY KEY,
        type CHAR(50) NOT NULL
    ) ENGINE=InnoDB;
    """,
    
    "data": """
    CREATE TABLE IF NOT EXISTS data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        total_cases INT(30) NOT NULL,
        total_deaths INT(30) NOT NULL,
        new_cases INT(30) NOT NULL,
        new_deaths INT(30) NOT NULL,
        id_location INT(30) NOT NULL,
        id_pandemie INT(30) NOT NULL,
        id_calendar INT(30) NOT NULL,
        FOREIGN KEY (id_calendar) REFERENCES calendar(id),
        FOREIGN KEY (id_location) REFERENCES location(id),
        FOREIGN KEY (id_pandemie) REFERENCES pandemie(id)
    ) ENGINE=InnoDB;
    """
}

## 6. Création des tables dans la base de données

Création des tables dans la base de données MySQL.

In [18]:
# Création des tables dans la base de données
print("Création des tables dans la base de données...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql(creer_db=True)
    
    if connexion is not None and curseur is not None:
        # Désactivation temporaire des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        # Création des tables
        for table, requete in requetes_creation_tables.items():
            print(f"Création de la table {table}...")
            curseur.execute(requete)
            print(f"Table {table} créée avec succès.")
        
        # Réactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 1;")
        
        # Validation des modifications
        connexion.commit()
        print("\nToutes les tables ont été créées avec succès.")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("Connexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la création des tables: {e}")

Création des tables dans la base de données...
Base de données 'epiviz' créée ou sélectionnée avec succès.
Création de la table calendar...
Table calendar créée avec succès.
Création de la table location...
Table location créée avec succès.
Création de la table pandemie...
Table pandemie créée avec succès.
Création de la table data...
Table data créée avec succès.

Toutes les tables ont été créées avec succès.
Connexion fermée.


## 7. Vérification des tables créées

Vérification des tables créées dans la base de données MySQL.

In [19]:
# Vérification des tables créées
print("Vérification des tables créées...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération de la liste des tables
        curseur.execute("SHOW TABLES;")
        tables = curseur.fetchall()
        
        print(f"Tables dans la base de données '{config_db['database']}':")
        for table in tables:
            print(f"- {table[0]}")
        
        # Vérification de la structure des tables
        for table in [table[0] for table in tables]:
            print(f"\nStructure de la table {table}:")
            curseur.execute(f"DESCRIBE {table};")
            colonnes = curseur.fetchall()
            for colonne in colonnes:
                print(f"- {colonne[0]}: {colonne[1]}")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification des tables: {e}")

Vérification des tables créées...
Connexion à la base de données 'epiviz' établie avec succès.
Tables dans la base de données 'epiviz':
- calendar
- data
- location
- pandemie

Structure de la table calendar:
- id: int(11)
- date_value: int(11)

Structure de la table data:
- id: int(11)
- total_cases: int(30)
- total_deaths: int(30)
- new_cases: int(30)
- new_deaths: int(30)
- id_location: int(30)
- id_pandemie: int(30)
- id_calendar: int(30)

Structure de la table location:
- id: int(11)
- country: char(50)
- continent: char(50)

Structure de la table pandemie:
- id: int(11)
- type: char(50)

Connexion fermée.
